# Distance Extraction

Going through this notebook will result in computed distances between local process models for each of the provided local process model sets.

## Imports

In [8]:
import os

from subprocess import Popen

## Inputs

In [12]:
lpms_directory = os.path.abspath("./data/lpms") # this is the directory that includes the sets of local process models
scripts_directory = os.path.abspath("./scripts")
print(scripts_directory)

res_directory = os.path.abspath("results") # this it the directory where results will be outputted
res_traceextraction = os.path.join(res_directory, "")

C:\Users\peeva\My\code\my\CombiningLPMDandPMSM\scripts


## Pre-computation

Since multiple measures are based on the language of the models, we precompute the traces each model can replay.

In [18]:
tr_processes = []
for lpm_set_name in os.listdir(lpms_directory):
    lpms_set = os.path.join(lpms_directory, lpm_set_name)
    if os.path.isdir(lpms_set):
        res_lpmspecific = os.path.join(res_directory, lpm_set_name)
        if not os.path.exists(res_lpmspecific): os.mkdir(res_lpmspecific)
        tr_processes.append(Popen(["python", os.path.join(scripts_directory, "model_trace_extraction.py"), lpms_set, res_lpmspecific]))

try:
    exitcodes = [p.wait() for p in tr_processes]
    if not all(el == 0 for el in exitcodes):
        print("Some trace extraction scripts did not finish successfully: " + str(exitcodes))
except KeyboardInterrupt:
    [p.kill() for p in tr_processes]
    print("Some trace extraction scripts did not finish successfully.")

## Distance calculations

In [17]:
# extract distances
dist_processes = []
for lpm_set_name in os.listdir(lpms_directory):
    lpms_set = os.path.join(lpms_directory, lpm_set_name)
    if os.path.isdir(lpms_set):
        res_lpmspecific = os.path.join(res_directory, lpm_set_name)
        res_distances = os.path.join(res_lpmspecific, "distances")
        if not os.path.exists(res_distances): os.mkdir(res_distances)

        trace_lpmsspecific = os.path.join(res_lpmspecific, "model_traces.csv")

        dist_processes.append(Popen(["python", os.path.join(scripts_directory, "distance_calculation/model_transition_label_distance.py"), lpms_set, res_distances]))
        dist_processes.append(Popen(["python", os.path.join(scripts_directory, "distance_calculation/model_trace_distance.py"), trace_lpmsspecific, res_distances]))
        dist_processes.append(Popen(["python", os.path.join(scripts_directory, "distance_calculation/model_node_distance.py"), lpms_set, res_distances]))
        dist_processes.append(Popen(["python", os.path.join(scripts_directory, "distance_calculation/model_efg_distance.py"), trace_lpmsspecific, res_distances]))
        dist_processes.append(Popen(["python", os.path.join(scripts_directory, "distance_calculation/model_ged_distance.py"), lpms_set, res_distances]))

try:
    exitcodes = [p.wait() for p in dist_processes]
    if not all(el == 0 for el in exitcodes):
        print("Some distance calculation scripts did not finish successfully: " + str(exitcodes))
except KeyboardInterrupt:
    [p.kill() for p in dist_processes]
    print("Some distance calculation scripts did not finish successfully.")